In [ ]:
import pandas as pd
from pycaret.classification import *
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Cargar datos
df = pd.read_csv("caso4_salud_enfermedades.csv")

# 2. Feature Engineering sofisticado
df['obesidad'] = (df['imc'] > 30).astype(int)
df['azucar_alta'] = (df['ingesta_azucar'] > 100).astype(int)

# Variable ordinal para ejercicio
orden_ejercicio = {'Nunca': 0, '1-2 veces': 1, '3+ veces': 2}
df['ejercicio_freq_num'] = df['ejercicio_frecuencia'].map(orden_ejercicio)

# Interacción no lineal que suele indicar riesgo metabólico
df['riesgo_metabolico'] = df['imc'] * df['ingesta_azucar'] * (1 - df['ejercicio_freq_num'])

# Clasificación basada en KMeans (estrategia semi no supervisada para clusterizar riesgo)
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

X_cluster = df[['imc', 'ingesta_azucar', 'edad']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster_riesgo'] = kmeans.fit_predict(X_scaled)

# 3. Configurar PyCaret con todo el nuevo feature engineering
clf = setup(data=df,
            target='enfermedad_cronica',
            session_id=777,
            ignore_features=['ejercicio_frecuencia'],
            categorical_features=['cluster_riesgo'],
            polynomial_features=True,
            bin_numeric_features=['edad', 'ingesta_azucar', 'riesgo_metabolico'],
            feature_selection=True,
            feature_interaction=True,
            remove_multicollinearity=True,
            multicollinearity_threshold=0.8,
            silent=True)

# 4. Comparar modelos optimizando por AUC (curva ROC balanceada)
best_model = compare_models(sort='AUC')

# 5. Ajuste fino buscando explicar más sin sobreajuste
tuned = tune_model(best_model, optimize='AUC')

# 6. Interpretación con SHAP (importantísimo en salud)
interpret_model(tuned, plot='summary')

# 7. Crear predicción simulada para una feria de salud
nuevo_caso = pd.DataFrame({
    'edad': [52],
    'imc': [33],
    'fuma': [1],
    'ejercicio_frecuencia': ['Nunca'],
    'ingesta_azucar': [130],
    'obesidad': [1],
    'azucar_alta': [1],
    'ejercicio_freq_num': [0],
    'riesgo_metabolico': [33*130*1],
    'cluster_riesgo': [2]
})
resultado = predict_model(tuned, data=nuevo_caso)
print("Diagnóstico proyectado:", resultado)

# 8. Exportar modelo para clínica móvil
save_model(tuned, 'modelo_salud_cronico_movil')
